#PyTorch  and Gradient Descent

In [ ]:
import  torch

##Tensor
PyTorch is a library for processing tensor  . Tensor is a number , matrix,vector or any n-dimension array.  Lets create a tensor with a single number 

In [ ]:
t1=torch.tensor(4.)
t1


tensor(4.)

 4. is a shorthand for 4.0 .It is used to indicate to  python that we  want floating point atribute  

In [ ]:
t1.dtype

torch.float32

let's try more complex  tensors
(all  the values in a tensor are of the same data type)

In [ ]:
#vector(1-d tensor)
t2=torch.tensor([1.,2,3,4])
t2

tensor([1., 2., 3., 4.])

In [ ]:
#matrix
t3=torch.tensor([[2.,3],[3,4],[5,3]])
t3

tensor([[2., 3.],
        [3., 4.],
        [5., 3.]])

In [ ]:
#3-D matrix
t4=torch.tensor([
                 [[1,2],[2,3]],
                 [[2,3],[4,5]]
                 ])
t4

tensor([[[1, 2],
         [2, 3]],

        [[2, 3],
         [4, 5]]])

tensor  can have any number of dimensions and different lenght anong each dimension . we can expect the length along each dimension of a tensor using  .shape property

In [ ]:
t1.shape


torch.Size([])

In [ ]:
t2.shape

torch.Size([4])

In [ ]:
t3.shape

torch.Size([3, 2])

In [ ]:
t4.shape

torch.Size([2, 2, 2])

Note that it's not possible to create tensors with an improper shape.

In [ ]:
##t5=torch.tensor([[1,2,3],[1,2]])

A `ValueError` is thrown because the lengths of the rows `[1,2,3]` and `[1, 2]` don't match.

##Tensor operations and gradiants 
we can combine tensor with usaual arithematic operations


In [ ]:
x=torch.tensor(3.)
w=torch.tensor(4.,requires_grad=True)
b=torch.tensor(5.,requires_grad=True)
x,w,b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [ ]:
#arithematic operations
y=w*x+b
y

tensor(17., grad_fn=<AddBackward0>)

as axpected y value is 17 , so what makes pytorch (requires_grade) unique is that  we can automatically compute derivative of y w.r.t the tensor whose requires_grade are set to True.This feature is called automatic gradient.
(dy/dw=0,dy/db=1)

- to compute the derivatives, we can invoke .backward method on your result y

In [ ]:
y.backward()

the derivatves of y w.r.t the input  tensor are stored in .grad property of the respective tensors

In [ ]:
print('dy/dx',x.grad)
print('dy/dw',w.grad)
print('dy/db',b.grad)

dy/dx None
dy/dw tensor(3.)
dy/db tensor(1.)


As expected, `dy/dw` has the same value as `x`, i.e., `3`, and `dy/db` has the value `1`. Note that `x.grad` is `None` because `x` doesn't have `requires_grad` set to `True`. 

The "grad" in `w.grad` is short for _gradient_, which is another term for derivative. The term _gradient_ is primarily used while dealing with vectors and matrices.

##Tensor Functions 
apart from arithematic operations ,the tourch module contain many number of functons   for creating and manupalating tensors .Let's look at some eg 

In [ ]:
#creating a tensor with fixed values for every element 
t6=torch.full((3,2),42)
t6

tensor([[42, 42],
        [42, 42],
        [42, 42]])

In [ ]:
#concatenating two tensor of compatable shape
t7=torch.cat((t3,t6))
t7

tensor([[ 2.,  3.],
        [ 3.,  4.],
        [ 5.,  3.],
        [42., 42.],
        [42., 42.],
        [42., 42.]])

In [ ]:
#computing the sin of each element
t8=torch.sin(t7)
t8

tensor([[ 0.9093,  0.1411],
        [ 0.1411, -0.7568],
        [-0.9589,  0.1411],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165]])

In [ ]:
#changing the shape of tensor
t9=t8.reshape(3,2,2)
t9

tensor([[[ 0.9093,  0.1411],
         [ 0.1411, -0.7568]],

        [[-0.9589,  0.1411],
         [-0.9165, -0.9165]],

        [[-0.9165, -0.9165],
         [-0.9165, -0.9165]]])

## Interoperability with Numpy

[Numpy](http://www.numpy.org/) is a popular open-source library used for mathematical and scientific computing in Python. It enables efficient operations on large multi-dimensional arrays and has a vast ecosystem of supporting libraries, including:

* [Pandas](https://pandas.pydata.org/) for file I/O and data analysis
* [Matplotlib](https://matplotlib.org/) for plotting and visualization
* [OpenCV](https://opencv.org/) for image and video processing


Instead of reinventing the wheel, PyTorch interoperates well with Numpy to leverage its existing ecosystem of tools and libraries.

In [ ]:
import numpy as np
x=np.array([[1,2],[2,3]])
x

array([[1, 2],
       [2, 3]])

we can convert numpy array to pytroch tensor using torch.from_numpy

In [ ]:
y=torch.from_numpy(x)
y

tensor([[1, 2],
        [2, 3]])

In [ ]:
x.dtype, y.dtype

(dtype('int64'), torch.int64)

In [ ]:
##we can convert pytorch to numpy array using .numpy method

In [ ]:
z=y.numpy()
z

array([[1, 2],
       [2, 3]])

The interoperability between PyTorch and Numpy is essential because most datasets you'll work with will likely be read and preprocessed as Numpy arrays.

You might wonder why we need a library like PyTorch at all since Numpy already provides data structures and utilities for working with multi-dimensional numeric data. There are two main reasons:

1. **Autograd**: The ability to automatically compute gradients for tensor operations is essential for training deep learning models.
2. **GPU support**: While working with massive datasets and large models, PyTorch tensor operations can be performed efficiently using a Graphics Processing Unit (GPU). Computations that might typically take hours can be completed within minutes using GPUs.



#Gradiant desent and Linear Regression

## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

In [ ]:
import numpy as np
import torch

## Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [ ]:
#input (temp,rainfall, humidity)
inputs=np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [ ]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

we create  separate  input and target values ,as we'll operate on them seperately . Also we have created numpy array as we have to load /read the csv files and do some processing and then we'll convert it to pytorch  tensor 

In [ ]:
#convert input and target to tensor 
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


##Linear regression Model from scratch
the wieghts and baises(w1,w2,w3.........b1 &b2) can also be represented as matrix ,initialized as random values. The first row of w and 1st element of b are used to pridect the 1st target variables i.e yeild of apples and oranges 

In [ ]:
w=torch.randn(2,3,requires_grad=True )#2 row and 3 column matrix
b=torch.randn(2,requires_grad=True)# biases b1,b2 form a vector
print(w)
print(b)

tensor([[-0.6509,  0.5736,  0.7497],
        [-1.1475, -0.2892,  0.3951]], requires_grad=True)
tensor([-0.1231,  1.0086], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [ ]:
inputs #temp,rainfall,humidity

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [ ]:
#for matrix multiplcation in pytroch we use @  and for transpose of matrix we use .t()
inputs @ w.t() + b             #  mx+c (m=inputs,x=w,c=b)  ,,, this will produ

tensor([[  23.0273,  -85.1475],
        [  39.0996, -103.5787],
        [  63.5911, -114.6638],
        [ -14.1139, -113.8550],
        [  62.5067,  -78.2763]], grad_fn=<AddBackward0>)

In [ ]:
def model(x):
  return x @ w.t() +b # here have created a function (model) which takes the input (x) 


`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [ ]:
#genarate pridictions 
preds= model(inputs)
print(preds)

tensor([[  23.0273,  -85.1475],
        [  39.0996, -103.5787],
        [  63.5911, -114.6638],
        [ -14.1139, -113.8550],
        [  62.5067,  -78.2763]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [ ]:
#compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

In [ ]:
diff=preds-targets 
diff #as there are some -ve values so wi'll take  the square of it(diff *diff) and find its average 
#to  find find avg we do the sum( torch.sum()) and divide it it total numbber of values of the given matrix(.numel())

tensor([[ -32.9727, -155.1476],
        [ -41.9004, -204.5787],
        [ -55.4089, -247.6638],
        [ -36.1139, -150.8550],
        [ -40.4933, -197.2763]], grad_fn=<SubBackward0>)

In [ ]:
torch.sum(diff*diff) / diff.numel() # this number tells us  that how bad is our model

tensor(19779.2695, grad_fn=<DivBackward0>)

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [ ]:
#MSE  loss
def mse(t1,t2):
  diff=t1-t2
  return torch.sum(diff*diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [ ]:
#compute loss
loss= mse(preds,targets)
print(loss)# the lower thw loss the better the  model 

tensor(19779.2695, grad_fn=<DivBackward0>)


In [ ]:
#compute gradiant 
loss.backward()

the gradiant is stored in .grad property of the respective tensor. Note that  the derivative of loss w.r.t the weights matrix is itself a matrix with the same dimension  

In [ ]:
#graident for weights 
print(w)
print(w.grad)

tensor([[-0.6509,  0.5736,  0.7497],
        [-1.1475, -0.2892,  0.3951]], requires_grad=True)
tensor([[ -3503.6367,  -3752.2927,  -2296.7837],
        [-16097.6943, -17402.0117, -10703.9727]])


In [ ]:
print(b)
print(b.grad)

tensor([-0.1231,  1.0086], requires_grad=True)
tensor([ -41.3779, -191.1043])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [ ]:
print(w)
w.grad
w-w.grad*1e-5

tensor([[-0.6509,  0.5736,  0.7497],
        [-1.1475, -0.2892,  0.3951]], requires_grad=True)


tensor([[-0.6159,  0.6111,  0.7726],
        [-0.9865, -0.1152,  0.5022]], grad_fn=<SubBackward0>)

In [ ]:
with torch.no_grad():
  w-=w.grad * 1e-5
  b-=b.grad * 1e-5

we multiply gradiant with a  very small number(10^5 in this case) .to ensure that we don't modify the weight by a very large amount . we want to take a small step in the downhill direction of the graident ,not a gaint leap.this number is call the learning rate of algorithum.

we use torch.no_grad() to indicate to pytorch that we shoud'nt track,calculate or modify the graident while updating the weights and baises 

In [ ]:
w,b # new weights and baises 


(tensor([[-0.6159,  0.6111,  0.7726],
         [-0.9865, -0.1152,  0.5022]], requires_grad=True),
 tensor([-0.1226,  1.0105], requires_grad=True))

In [ ]:
#let's verify the loss is actually lower
preds=model(inputs)
loss= mse(preds,targets)
print(loss)


tensor(13331.3496, grad_fn=<DivBackward0>)


before the loss  was 19779.2695 and now it is 13331.3496 ... there is so much decrease .. and that's preety good 


befoe we proceed. we rest the gradiantb to zero by invoking .zero() method. we need to do this because pytorch accumulates gradiants . otherwise the next if we invoke .backward on the loss , the new gradiant value is added to the existing gradiant , which will lead to unexpected results 

In [ ]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


#Train the model using gradiant decent 
as we saw above we reduse the loss and improve our model using the gradiant desent optimization algorithum .
thus we can train our model using following steps :


1.   Generate pridiction 
2.   Calculate the loss
3.   compute gradiant w.r.t. weights and baises 
4.   adjust the weights by substracting a smallquantity proportional to the gradiant 
5. rest the gradiant to zero

lets implement the above steps 



In [ ]:
# generate pridictions
preds=model(inputs)
preds

tensor([[ 29.0870, -57.1323],
        [ 47.0603, -66.7636],
        [ 72.9998, -71.1299],
        [ -8.0765, -85.9902],
        [ 70.1346, -42.9682]], grad_fn=<AddBackward0>)

In [ ]:
#Calculate the loss
loss=mse(preds,targets)
print(loss)

tensor(13331.3496, grad_fn=<DivBackward0>)


In [ ]:
#compute gradiant w.r.t. weights and baises
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -2878.1399,  -3080.4521,  -1882.1649],
        [-13205.4492, -14294.3984,  -8786.3008]])
tensor([ -33.9590, -156.7968])


lets update the weights and baises using the gradiant computed above

In [ ]:
#adjust the weights by substracting a smallquantity proportional to the gradiant
with torch.no_grad():
  w-=w.grad* 1e-5
  b-=b.grad* 1e-5
  w.grad.zero_()
  b.grad.zero_()



lets take a look at new weights and baises 

In [ ]:
print(w)
print(b)

tensor([[-0.5871,  0.6419,  0.7915],
        [-0.8545,  0.0277,  0.5900]], requires_grad=True)
tensor([-0.1223,  1.0120], requires_grad=True)


with new weights and baises , the model should have lower loss 

In [ ]:
#calculate loss
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(8986.1738, grad_fn=<DivBackward0>)


we have reached a significant reduction in the loss merly by adjusting the weights and baises slightly using gradiant decent 

#Train for multiple epochs 
to reduse the loss further, we can repate the process of adjusting the weights and baises using gradiant decent multiple times. Each itrration is called epoch. Let's train the model 100 times 

In [ ]:
for i in range(500):
  preds=model(inputs)
  loss=mse(preds,targets)
  loss.backward
  with torch.no_grad():
    w-=w.grad *1e-5
    b-=b.grad *1e-5
    w.grad.zero_()
    b.grad.zero_()


lets verify the loss is now lower

In [ ]:
#calculate loss
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(8986.1738, grad_fn=<DivBackward0>)


In [ ]:
preds

tensor([[ 34.0616, -34.1354],
        [ 53.5951, -36.5428],
        [ 80.7236, -35.3890],
        [ -3.1195, -63.1215],
        [ 76.3956, -13.9819]], grad_fn=<AddBackward0>)

In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])